###  NBA Games

In [1]:
import os, sys
from glob import glob
import pandas as pd

raw_data = {}
for year in range(2000,2017):

    fp = os.path.join("../data", f"nba-{year}.csv")

    if not os.path.exists(fp):
        url = f"http://www.basketball-reference.com/leagues/NBA_{year}_games.html"
        tables = pd.read_html(url)
        games = tables[0]
        games.to_csv(fp)
        raw_data[str(year)] = games
    else:
        raw_data[str(year)] = pd.read_csv(fp)

In [2]:
raw_data['2008'].head()

,Unnamed: 0,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,0,"Tue, Oct 30, 2007",7:30 pm,Utah Jazz,117,Golden State Warriors,96,Box Score,NaN,19832,NaN
1,1,"Tue, Oct 30, 2007",7:30 pm,Houston Rockets,95,Los Angeles Lakers,93,Box Score,NaN,18997,NaN
2,2,"Tue, Oct 30, 2007",7:00 pm,Portland Trail Blazers,97,San Antonio Spurs,106,Box Score,NaN,18797,NaN
3,3,"Wed, Oct 31, 2007",8:00 pm,Dallas Mavericks,92,Cleveland Cavaliers,74,Box Score,NaN,20562,NaN
4,4,"Wed, Oct 31, 2007",8:30 pm,Seattle SuperSonics,103,Denver Nuggets,120,Box Score,NaN,19380,NaN


As you can see, we have some extra rows of mostly NaNs, the column names aren't useful, and we have some dtypes to fix up.

In [3]:
%%time 
seasons = {}
for year, games in raw_data.items():
    
    column_names = {'Date': 'date', 'Start (ET)': 'start',
                'Unamed: 2': 'box', 'Visitor/Neutral': 'away_team', 
                'PTS': 'away_points', 'Home/Neutral': 'home_team',
                'PTS.1': 'home_points', 'Unamed: 7': 'n_ot'}

    seasons[str(year)] = (games.rename(columns=column_names)
        .dropna(thresh=4)
        [['date', 'away_team', 'away_points', 'home_team', 'home_points']]
        .assign(date=lambda x: pd.to_datetime(x['date'], format='%a, %b %d, %Y'))
        .set_index('date', append=True)
        .rename_axis(["game_id", "date"])
        .sort_index())
    

CPU times: user 115 ms, sys: 3.16 ms, total: 118 ms
Wall time: 119 ms


In [4]:
seasons['2014']

,,away_team,away_points,home_team,home_points
game_id,date,,,,
0,2013-10-29,Orlando Magic,87,Indiana Pacers,97
1,2013-10-29,Los Angeles Clippers,103,Los Angeles Lakers,116
2,2013-10-29,Chicago Bulls,95,Miami Heat,107
3,2013-10-30,Brooklyn Nets,94,Cleveland Cavaliers,98
4,2013-10-30,Atlanta Hawks,109,Dallas Mavericks,118
5,2013-10-30,Washington Wizards,102,Detroit Pistons,113
6,2013-10-30,Los Angeles Lakers,94,Golden State Warriors,125
7,2013-10-30,Charlotte Bobcats,83,Houston Rockets,96
8,2013-10-30,Orlando Magic,115,Minnesota Timberwolves,120


- `dropna` has a thresh argument. If at least thresh items are missing, the row is dropped. We used it to remove the "Month headers" that slipped into the table.
- `assign` can take a callable. This lets us refer to the DataFrame in the previous step of the chain. Otherwise we would have to assign temp_df = games.dropna()... And then do the pd.to_datetime on that.
-`set_index` has an append keyword. We keep the original index around since it will be our unique identifier per game.
- `.rename_axis` is used to set the index names.

### How many days of rest did each team get between each game?


In this case, an observation is a (team, game) pair, which we don't have yet. Rather, we have two observations per row, one for home and one for away. We'll fix that with `pd.melt`.

`pd.melt` works by taking observations that are spread across columns (away_team, home_team), and melting them down into one column with multiple rows. However, we don't want to lose the metadata (like game_id and date) that is shared between the observations. By including those columns as id_vars, the values will be repeated as many times as needed to stay with their observations.

In [5]:
%%time
tidy = {}
for year, games in seasons.items():
    tidy[str(year)] = pd.melt(games.reset_index(),
               id_vars=['game_id', 'date'], value_vars=['away_team', 'home_team'],
               value_name='team')

CPU times: user 79.2 ms, sys: 2.47 ms, total: 81.7 ms
Wall time: 80.1 ms


In [6]:
tidy['2014'].groupby('team')['date'].diff().dt.days - 1

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     0.0
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19   -2.0
20   -2.0
21   -2.0
22    NaN
23    NaN
24    NaN
25   -2.0
26    NaN
27    NaN
28    NaN
29   -2.0
30    NaN
31    NaN
32    NaN
33    NaN
34    NaN
35    NaN
36    1.0
37    1.0
Name: date, dtype: float64

In [7]:
tidy['2010'].groupby('team')['date'].diff().dt.days - 1

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     0.0
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    2.0
19    NaN
20    1.0
21    NaN
22    NaN
23    NaN
24    NaN
25    1.0
26    NaN
27    1.0
28    NaN
29    NaN
     ... 
46   -3.0
47    NaN
48   -3.0
49    NaN
50   -3.0
51   -3.0
52   -2.0
53   -3.0
54   -2.0
55    1.0
56    1.0
57    1.0
58   -2.0
59   -2.0
60    1.0
61    2.0
62    1.0
63    1.0
64   -2.0
65    1.0
66   -2.0
67    1.0
68    1.0
69    3.0
70    2.0
71    2.0
72    0.0
73    0.0
74    2.0
75    0.0
Name: date, Length: 76, dtype: float64

In [8]:
tidy['2010']['rest'] = tidy['2010'].sort_values('date').groupby('team').date.diff().dt.days - 1

# References
* [Modern Pandas (Part 5): Tidy Data by T. Augspurger](https://tomaugspurger.github.io/modern-5-tidy)
